<a href="https://colab.research.google.com/github/ReidelVichot/PUS2022_Heat-TreesPHL/blob/main/philly_health_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import eli5
from eli5.sklearn import PermutationImportance
from scipy.stats import pearsonr

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
health = pd.read_csv('/content/drive/Shareddrives/PUS2022/PUS project/philly_health_data.csv')
health.describe()

,PlaceFIPS,TractFIPS,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,...,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev
count,376.0,3.760000e+02,376.000000,376.000000,376.000000,376.000000,376.000000,376.000000,376.000000,376.000000,...,376.000000,376.000000,376.000000,376.000000,376.000000,376.000000,376.000000,376.000000,376.000000,375.000000
mean,4260000.0,4.210103e+10,4058.377660,13.434309,25.765160,20.179521,34.295479,74.481383,5.809043,11.589096,...,3.420479,29.305319,80.857447,16.168617,33.205585,81.397606,14.893085,42.241489,4.093883,19.860800
std,0.0,9.958006e+04,1625.805143,5.792134,6.111099,4.587412,9.248421,7.225807,1.830824,1.809077,...,1.075289,8.057968,2.665253,3.726829,8.095460,2.848432,4.398158,5.865477,1.734325,8.798686
min,4260000.0,4.210100e+10,74.000000,4.100000,6.500000,8.500000,10.800000,33.500000,1.100000,8.200000,...,1.000000,12.800000,72.900000,8.200000,17.400000,65.100000,5.000000,28.800000,0.700000,3.800000
25%,4260000.0,4.210101e+10,2889.250000,9.175000,22.950000,16.500000,28.800000,71.900000,4.700000,10.100000,...,2.700000,23.450000,78.200000,13.400000,26.200000,79.700000,11.875000,37.000000,2.900000,12.600000
50%,4260000.0,4.210102e+10,3944.500000,12.700000,26.250000,19.300000,33.300000,75.300000,5.600000,11.400000,...,3.300000,29.150000,81.300000,16.000000,32.950000,81.600000,14.600000,42.850000,3.800000,19.000000
75%,4260000.0,4.210103e+10,5129.000000,16.200000,29.750000,23.400000,42.350000,79.425000,6.600000,13.100000,...,4.200000,35.625000,83.300000,19.000000,40.600000,83.500000,18.200000,47.200000,5.200000,26.600000
max,4260000.0,4.210199e+10,8322.000000,35.300000,50.900000,33.600000,56.400000,89.600000,18.700000,16.300000,...,7.100000,46.200000,86.000000,25.800000,48.800000,86.000000,24.800000,52.600000,10.300000,42.400000


In [48]:
shapefile = gpd.read_file("/content/drive/Shareddrives/PUS2022/PUS project/phil_tracts_10_1_2020/phil_tracts_10_1_2020.shx")
shapefile

,GEOID,ALAND,pop,male,female,white,black,nat_amr,asian,pac_ilndr,...,owned_home,rented_hom,disability,dis_noheal,recieved_s,below_pov,unemployed,no_health,med_inc,geometry
0,42101012204,879459,3873,0.429383,0.570617,0.161890,0.729925,0.005680,0.051123,0.000000,...,0.527568,0.472432,0.179447,0.071942,0.145751,0.183323,0.042991,0.046097,32356,"POLYGON ((-75.22066 40.00394, -75.21923 40.004..."
1,42101012203,227696,791,0.608091,0.391909,0.204804,0.179520,0.000000,0.343869,0.092288,...,0.000000,1.000000,0.084703,0.000000,0.037523,0.296443,0.015306,0.048040,51610,"POLYGON ((-75.21389 40.00549, -75.21387 40.005..."
2,42101013602,246812,4031,0.498884,0.501116,0.775738,0.046142,0.000000,0.048871,0.000000,...,0.480769,0.519231,0.047631,0.041667,0.032525,0.064817,0.019526,0.031079,57500,"POLYGON ((-75.18637 39.97354, -75.18604 39.973..."
3,42101034502,1021734,5427,0.442970,0.557030,0.595909,0.196610,0.000000,0.163626,0.003132,...,0.378373,0.621627,0.182605,0.085772,0.255080,0.153486,0.030519,0.105215,30840,"POLYGON ((-75.04729 40.08293, -75.04652 40.083..."
4,42101000902,155167,2764,0.467077,0.532923,0.689219,0.054631,0.017728,0.175832,0.000000,...,0.215617,0.784383,0.174747,0.000000,0.295311,0.136202,0.009169,0.023155,43125,"POLYGON ((-75.16017 39.94686, -75.16014 39.947..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,42101002802,362691,5772,0.512301,0.487699,0.644144,0.088358,0.000000,0.234061,0.000000,...,0.583098,0.416902,0.132883,0.208605,0.034422,0.177712,0.022441,0.165800,31703,"POLYGON ((-75.16707 39.92680, -75.16693 39.926..."
404,42101002701,224842,4098,0.538067,0.461933,0.698878,0.104441,0.008053,0.104685,0.000000,...,0.527143,0.472857,0.085163,0.186246,0.179933,0.130733,0.084972,0.125427,41288,"POLYGON ((-75.15600 39.92553, -75.15592 39.925..."
405,42101980600,1286762,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.18314 39.90775, -75.18308 39.908..."
406,42101036501,1816210,4799,0.466764,0.533236,0.796833,0.112107,0.001042,0.060846,0.000000,...,0.557265,0.442735,0.196083,0.034006,0.161694,0.147530,0.042545,0.059578,32307,"POLYGON ((-75.02371 40.13005, -75.02301 40.130..."


In [49]:
def gettract(cname):
  return cname.split('-')[-1]
health["GEOID"] = health.Place_TractID.astype(str).map(gettract)
health['GEOID1'] = health['GEOID'].astype(int)

In [50]:
dat = shapefile.merge(health, left_on = 'GEOID', right_on = 'GEOID1')
dat.head()

ValueError: ignored